In [1]:
import pandas as pd
import numpy as np
import warnings

from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'

warnings.filterwarnings("ignore", category=RuntimeWarning) 
pd.options.mode.chained_assignment = None  # default='warn'

%matplotlib inline

In [2]:
data = pd.read_csv(path+'data/ml_input.csv')
merged = pd.read_csv(path+'data/processed.csv')
parameters = pd.read_csv(path+'parameters/neutralnetworkclassifier.csv')

In [3]:
N = 2021

In [4]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season < N]
X_train = train.drop(['season', 'round', 'podium', 'driver_points_from'], axis=1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)

In [5]:
data.columns

Index(['season', 'round', 'podium', 'q_delta', 'starting_grid',
       'driver_points_before', 'constructor_points_before',
       'driver_points_from', 'driver_points_per', 'points_percentage',
       ...
       'circuit_id_rodriguez', 'circuit_id_sepang', 'circuit_id_shanghai',
       'circuit_id_silverstone', 'circuit_id_sochi', 'circuit_id_spa',
       'circuit_id_suzuka', 'circuit_id_villeneuve', 'circuit_id_yas_marina',
       'circuit_id_zandvoort'],
      dtype='object', length=116)

In [6]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['season', 'round', 'podium', 'driver_points_from'], axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace=True, drop=True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == N]['round'].unique().max()
    return model_score

In [7]:
def get_predictions(model):
    predictions = []
    for circuit in df[df.season == N]['round'].unique():

        test = df[(df.season == N) & (df['round'] == circuit)]
        X_test = test.drop(['season', 'round', 'podium', 'driver_points_from'], axis=1)
        y_test = test.podium
        
        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
        
        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns=['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop=True)
        prediction_df['predicted'] = prediction_df.proba_1.map(lambda x: 1 if float(x) == float(prediction_df.proba_1.max()) else 0)
        

        predictions += list(prediction_df.predicted.values)

    return predictions

In [8]:
parameters.head()

,Unnamed: 0,model,hidden_layer_sizes,activation,solver,alpha,score
0,194,neural_network_classifier,"(80, 20, 40, 5)",relu,lbfgs,2.636651,0.500000
1,360,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.000100,0.545455
2,366,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.007848,0.500000
3,367,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.016238,0.500000
4,370,neural_network_classifier,"(75, 25, 50, 10)",tanh,lbfgs,0.143845,0.500000


In [9]:
params = parameters.iloc[1]
hidden_layer_sizes = eval(params.hidden_layer_sizes)
activation = params.activation
solver = params.solver
alpha = params.alpha

params

Unnamed: 0                                  360
model                 neural_network_classifier
hidden_layer_sizes             (75, 25, 50, 10)
activation                                 tanh
solver                                    lbfgs
alpha                                    0.0001
score                                  0.545455
Name: 1, dtype: object

In [10]:
model = MLPClassifier(
    hidden_layer_sizes=hidden_layer_sizes,
    activation=activation, 
    solver=solver, alpha=alpha, 
    random_state=1)

model.fit(X_train, y_train)

model_score = score_classification(model)
predictions = get_predictions(model)

print(model_score)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.5454545454545454


In [11]:
test = merged.query('season ==@N')
test['predicted'] = predictions
test = test[['season', 'round', 'circuit_id', 'driver', 'starting_grid' ,'podium', 'predicted']]

In [12]:
test.query('predicted == 1')

,season,round,circuit_id,driver,starting_grid,podium,predicted
2742,2021,1,bahrain,lewis_hamilton,2,1,1
2763,2021,2,imola,lewis_hamilton,1,2,1
2781,2021,3,portimao,lewis_hamilton,2,1,1
2801,2021,4,catalunya,lewis_hamilton,1,1,1
2821,2021,5,monaco,max_verstappen,2,1,1
2840,2021,6,baku,sergio_perez,6,1,1
2859,2021,7,ricard,lewis_hamilton,2,2,1
2876,2021,8,red_bull_ring,max_verstappen,1,1,1
2896,2021,9,red_bull_ring,max_verstappen,1,1,1
2917,2021,11,hungaroring,lewis_hamilton,1,2,1
